In [1]:
import azureml
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.run import Run
import os
import glob2 as glob
import tensorflow as tf
from tensorflow.keras import models, layers, callbacks, optimizers
import numpy as np
import pickle
import random
#from preprocessing import preprocess_depthmap, preprocess_targets
import argparse
import cv2
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import math

### Mount the workspace and data

In [ ]:
from azureml.core import Workspace, Dataset

subscription_id = '9b82ecea-6780-4b85-8acf-d27d79028f07'
resource_group = 'cgm-ml-prod'
workspace_name = 'cgm-azureml-prod'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='anon-depthmap-npy-test')

In [ ]:
with dataset.mount() as mount_context:
       # list top level mounted files and folders in the dataset
        print(os.listdir(mount_context.mount_point))
print(mount_context.mount_point) 
#get the location of the mountpoint for your machine

In [ ]:
mount_context = dataset.mount()
mount_context.start()  # this will mount the file streams
print(mount_context.mount_point)

In [3]:
# Get the image target size and define other hyperparameters
# target_size = args.target_size[0].split("x")
image_target_width = 172
image_target_height = 224
# Get batch size and epochs
batch_size = 2
epochs = 50

# Get the current run.
# run = Run.get_context()

In [ ]:
# Get dataset.
dataset_path = "../../../../../../npy"

In [ ]:
# Get the QR-code paths.
print("Dataset path:", dataset_path)
print(glob.glob(os.path.join(dataset_path, "*"))) # Debug
print("Getting QR-code paths...")

qrcode_paths = glob.glob(os.path.join(dataset_path, "*"))

### Split the data for training and validation

In [ ]:
## Here only small dataset is used, bascially two scans so dividing the whole training and validation into 50%. Genral good practise 
## is 80-20%.
split_index = int(len(qrcode_paths)*.5)
qrcode_paths_training = qrcode_paths[:split_index]
qrcode_paths_validate = qrcode_paths[split_index:]

In [ ]:
## This is dataset is used for visualising the activation maps. Due to small dataset , i am using training data as activation data.  
activation_data = qrcode_paths_training[0]
activation = [activation_data]
print(qrcode_paths_training)
print(activation_data)

In [ ]:
def get_depthmap_files(paths):
    pickle_paths = []
    for path in paths:
        pickle_paths.extend(glob.glob(os.path.join(path, "**", "*.p")))
    return pickle_paths

In [ ]:
paths_training = get_depthmap_files(qrcode_paths_training)
paths_validate = get_depthmap_files(qrcode_paths_validate)
paths_activation = get_depthmap_files(activation_data)
print("Using {} files for training.".format(len(paths_training)))
print("Using {} files for validation.".format(len(paths_validate)))
print("using {} files for activation.".format(len(paths_activation)))

In [ ]:
# Function for loading and processing depthmaps.
def tf_load_pickle(path):

    def py_load_pickle(path):
        depthmap, targets = pickle.load(open(path.numpy(), "rb"))
        depthmap = preprocess_depthmap(depthmap)
        depthmap = tf.image.resize(depthmap, (image_target_height, image_target_width))
        targets = preprocess_targets(targets, targets_indices)
        return depthmap, targets

    depthmap, targets = tf.py_function(py_load_pickle, [path], [tf.float32, tf.float32])
    depthmap.set_shape((image_target_height, image_target_width, 1))
    targets.set_shape((len(targets_indices,)))
    return depthmap, targets

def tf_flip(image):

    image = tf.image.random_flip_left_right(image)
    return image

In [ ]:
# Parameters for dataset generation.
shuffle_buffer_size = 2
subsample_size = 1
channels = list(range(0, 3))
targets_indices = [0] # 0 is height, 1 is weight.

# Create dataset for training.
paths = paths_training
dataset = tf.data.Dataset.from_tensor_slices(paths)
dataset = dataset.map(lambda path: tf_load_pickle(path))
dataset = dataset.cache()
dataset = dataset.shuffle(shuffle_buffer_size)
dataset = dataset.map(lambda image, label: (tf_flip(image), label))
#dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
dataset_training = dataset

In [ ]:
# Create dataset for activation maps.
paths = paths_activation
dataset = tf.data.Dataset.from_tensor_slices(paths)
dataset = dataset.map(lambda path: tf_load_pickle(path))
dataset = dataset.cache()
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
dataset_check = dataset

In [ ]:
# Create dataset for activation maps.
paths = paths_validate
dataset = tf.data.Dataset.from_tensor_slices(paths)
dataset = dataset.map(lambda path: tf_load_pickle(path))
dataset = dataset.cache()
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
dataset_validate = dataset

In [ ]:
# Create dataset for validation.
# Note: No shuffle necessary.
paths = paths_validate
dataset = tf.data.Dataset.from_tensor_slices(paths)
dataset = dataset.map(lambda path: tf_load_pickle(path))
dataset = dataset.cache()
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
dataset_validate = dataset

In [4]:
# Instantiate model.
model = models.Sequential()

model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), padding="same", activation="relu", input_shape=(image_target_height, image_target_width, 1)))
model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

#model.add(layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
#model.add(layers.Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
#model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.25))
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1, activation="linear"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 172, 16)      160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 172, 16)      2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 86, 16)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 86, 32)       4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 86, 32)       9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 43, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 43, 64)       

In [5]:
print(type(model))

<class 'tensorflow.python.keras.engine.sequential.Sequential'>


In [ ]:
## callbacks for training 
training_callbacks = []
best_model_path = "best_model.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=best_model_path,
    monitor="val_loss", 
    save_best_only=True,
    verbose=1
)

### Gradcam class to compute the heatmaps for the given layer

In [ ]:
class GradCAM:
    def __init__(self, model, layerName):
        self.model = model
        self.layerName = layerName
    
        self.gradModel = Model(inputs=[self.model.inputs], 
                                            outputs=[self.model.get_layer(self.layerName).output, model.output])
    
    def compute_heatmap(self, image, classIdx, eps=1e-8):
    
        with tf.GradientTape() as tape:
            tape.watch(self.gradModel.get_layer(self.layerName).output)
            inputs = tf.cast(image, tf.float32)
            (convOutputs,predictions) = self.gradModel(inputs)
            if len(predictions)==1:
                loss = predictions[0]
            else:
                loss = predictions[:, classIdx]

        grads = tape.gradient(loss, convOutputs)
    
        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads

        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]

        weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)

        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))

        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("float32")
        return heatmap
        
    
#     def overlay_heatmap(self, heatmap, image, alpha=0.5, colormap=cv2.COLORMAP_HOT):
#         heatmap = cv2.applyColorMap(heatmap, colormap)
#         output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)
    
#         return (heatmap, output)

### Function to create gridmap for the given set of images

In [ ]:
def make_grid(image_dir):
    from glob import glob
    files = glob(image_dir+'/*.png')    
    result_figsize_resolution = 80 # 1 = 100px

    # images_list = os.listdir(images_dir)
    images_count = 8
    # Calculate the grid size:
    grid_size = math.ceil(math.sqrt(images_count))

    # Create plt plot:
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(result_figsize_resolution, result_figsize_resolution))
    current_file_number = 0
    samples = files[:images_count]
    for image in samples:
        x_position = current_file_number % grid_size
        y_position = current_file_number // grid_size
        plt_image = plt.imread(image)
        axes[x_position, y_position].imshow(plt_image)
    # print((current_file_number + 1), '/', images_count, ': ', image_filename)

        current_file_number += 1

    plt.subplots_adjust(left=0.0, right=1.0, bottom=0.0, top=1.0)
    save_location = '{}/grid'.format(image_dir)
    if not os.path.exists(save_location):            
        os.makedirs(save_location)
    plt.savefig('{}/resultgrid.png'.format(save_location))
    plt.clf()
    for file in files:
        os.remove(file)

### Keras callback for the gradcam visualisation

In [ ]:
class GRADCamLogger(tf.keras.callbacks.Callback):
    def __init__(self, activation_data, layer_name):
        super(GRADCamLogger, self).__init__()
        self.activation_data = activation_data
        self.layer_name = layer_name
        

    def on_epoch_end(self,epoch,logs):
        images = []
        grad_cam = []
      ## Initialize GRADCam Class
        cam = GradCAM(self.model, self.layer_name)
        count =0
        foldername = 'out/epoch{}'.format(epoch)
        if not os.path.exists(foldername):            
            os.makedirs(foldername)           
        for data in self.activation_data:
            image = data[0]
            image = np.expand_dims(image, 0)
            pred = model.predict(image)
            classIDx = np.argmax(pred[0])
  
        ## Compute Heatmap
            heatmap = cam.compute_heatmap(image, classIDx)
            image = image.reshape(image.shape[1:])
            image = image*255
            image = image.astype(np.uint8)

        ## Overlay heatmap on original image
            heatmap = cv2.resize(heatmap, (image.shape[1],image.shape[0]))
            implot = plt.imshow(np.squeeze(image))            
            plt.imshow(heatmap,alpha=.6,cmap='inferno')
            plt.axis('off')
            plt.savefig('out/epoch{}/out{}.png'.format(epoch,count), bbox_inches='tight', transparent=True,pad_inches=0)
            plt.clf()
            count+=1
        make_grid(foldername)
        

In [ ]:
##Adding Gradcam(Activation maps) callbacks into training  
training_callbacks  =[]
layer_name = 'conv2d_11'
cam_callback = GRADCamLogger(dataset_check,layer_name)
model.compile(
    optimizer="nadam",
    loss="mse",
    metrics=["mae"]
)
training_callbacks.append(checkpoint_callback)
training_callbacks.append(cam_callback)

In [ ]:
## training the model
model.fit(
    dataset_training.batch(batch_size),
    validation_data=dataset_validate.batch(batch_size),
    epochs=epochs,
    callbacks=training_callbacks
)

### convert the gridmaps pngs into gif for visualisation

In [ ]:
import glob2
import imageio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

data = 'out'
grid_data = glob2.glob('out/**/*.png')
images =[]
for filename in grid_data:
    images.append(imageio.imread(filename))
imageio.mimsave('movie.gif', images)
img = mpimg.imread("movie.gif")
plt.imshow(img)

In [ ]:
## stop the mont data
mount_context.stop()